In [1]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from model import seq2seqParams, Seq2SeqModel
from dataset import  load_xmc_seq2seq_dataset_from_disk

In [2]:
dataset_name = 'eurlex-4k'
model_name = 'bart-large'
params_path = f'./params/{dataset_name}/{model_name}.json'
data_dir = "xmc-base/eurlex-4k"

In [3]:
train_dataset, val_dataset = load_xmc_seq2seq_dataset_from_disk(data_dir)
args = seq2seqParams.load_config(params_path)
model =  Seq2SeqModel(args)

Loading datasets from xmc-base/eurlex-4k


In [4]:
task = 'train'
if task == 'train':
    output_dir = f"output/{dataset_name}/{model_name}"
    trainer, _ = model.gen_train(train_dataset, val_dataset,output_dir)
else:
    output_dir = f"output/{dataset_name}/{model_name}"
    model.model.from_pretrained(output_dir)

/root/miniconda3/envs/kg/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training model facebook/bart-large with output directory output/eurlex-4k/bart-large
Training arguments: seq2seqParams(model_name_or_path='facebook/bart-large', max_input_length=256, max_output_length=80, num_train_epochs=5, batch_size=1, learning_rate=5e-05, use_fp16=True, logging_steps=20, weight_decay=0.01, warmup_ratio=0.1, use_tensorboard=True, num_beams=4)


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 10.57 GiB of which 13.12 MiB is free. Process 466507 has 3.46 GiB memory in use. Process 641853 has 7.10 GiB memory in use. Of the allocated memory 6.81 GiB is allocated by PyTorch, and 98.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [8]:
prefix = 'Summary this paragraph using unstemmed words: '
X_tst_text = load_texts(data_dir+"/X.tst.txt")
X_tst_text = [prefix + text for text in X_tst_text]

model.predict(X_tst_text,batch_size=4,max_input_length = args.max_input_length,max_output_length=80,num_beams=args.num_beams,output_path=data_dir)


Generating:   0%|          | 0/967 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generating: 100%|██████████| 967/967 [07:12<00:00,  2.23it/s]


['cattle,france,market support,regulation of agricultural production',
 'european community,market stabilisation,shipbuilding,south korea',
 'export refund,fishery product',
 'beef,hungary,import licence,originating product,tariff quota',
 'animal disease,community aid,fish,health control,veterinary inspection',
 'combined nomenclature,common organisation of markets,sugar',
 'advertising malpractice,approximation of laws,consumer protection,dissemination of information,intra-community trade',
 'branch of activity,community statistics,company structure,competitiveness,nomenclature',
 'carcass,cattle,classification,meat processing industry',
 'aid to agriculture,eaggf guarantee section,eu country,rural development,state aid',
 'export licence,export refund,milk product',
 'atmospheric pollution,ozone,pollution control measures',
 'aid to agriculture,community aid,producer group,production aid,tropical fruit',
 'animal disease,community aid,financial aid,germany,health control,indemnifica